In [1]:
import requests
from IPython.display import Markdown
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import altair as alt
from requests.utils import quote
import os
from datetime import timedelta
from mod import alt_theme

In [2]:
fmt = "{:%Y-%m-%d}"

# Can optionally use number of days to choose dates
n_days = 60
end_date = fmt.format(pd.datetime.today())
start_date = fmt.format(pd.datetime.today() - timedelta(days=n_days))

renderer = "kaggle"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]
bot_names = ["stale", "codecov", "jupyterlab-dev-mode", "henchbot"]

In [3]:
# Parameters
renderer = "kaggle"


In [4]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [5]:
# Discourse API key
api = {'Api-Key': os.environ['DISCOURSE_API_KEY'],
       'Api-Username': os.environ['DISCOURSE_API_USERNAME']}

In [6]:
# Discourse
def topics_to_markdown(topics, n_list=10):
    body = []
    for _, topic in topics.iterrows():
        title = topic['fancy_title']
        slug = topic['slug']
        posts_count = topic['posts_count']
        url = f'https://discourse.jupyter.org/t/{slug}'
        body.append(f'* [{title}]({url}) ({posts_count} posts)')
    body = body[:n_list]
    return '\n'.join(body)

def counts_from_activity(activity):
    counts = activity.groupby('category_id').count()['bookmarked'].reset_index()
    counts['parent_category'] = None
    for ii, irow in counts.iterrows():
        if parent_categories[irow['category_id']] is not None:
            counts.loc[ii, 'parent_category'] = parent_categories[irow['category_id']]

    counts['category_id'] = counts['category_id'].map(lambda a: category_mapping[a])
    counts['parent_category'] = counts['parent_category'].map(lambda a: category_mapping[a] if a is not None else 'parent')
    is_parent = counts['parent_category'] == 'parent'
    counts.loc[is_parent, 'parent_category'] = counts.loc[is_parent, 'category_id'] 
    counts['parent/category'] = counts.apply(lambda a: a['parent_category']+'/'+a['category_id'], axis=1)
    counts = counts.sort_values(['parent_category', 'bookmarked'], ascending=False)
    return counts

# Community forum activity

The [Jupyter Community Forum](https://discourse.jupyter.org) is a place for Jovyans across the
community to talk about Jupyter tools in interactive computing and how they fit into their
workflows. It's also a place for developers to share ideas, tools, tips, and help one another.

Below are a few updates from activity in the Discourse. For more detailed information about
the activity on the Community Forum, check out these links:

* [The users page](https://discourse.jupyter.org/u) has information about user activity
* [The top posts page](https://discourse.jupyter.org/top) contains a list of top posts, sorted
  by various metrics.

In [7]:
# Get categories for IDs
url = "https://discourse.jupyter.org/site.json"
resp = requests.get(url, headers=api)
category_mapping = {cat['id']: cat['name'] for cat in resp.json()['categories']}
parent_categories = {cat['id']: cat.get("parent_category_id", None) for cat in resp.json()['categories']}

In [8]:
# Base URL to use
url = "https://discourse.jupyter.org/latest.json"

## Topics with lots of likes

"Likes" are a way for community members to say thanks for a helpful post, show their
support for an idea, or generally to share a little positivity with somebody else.
These are topics that have generated lots of likes in recent history.

In [9]:
params = {"order": "likes", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
liked = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(liked))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (103 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (59 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (29 posts)
* [JupyterHub, Binder workshop in Oslo, Sept. 5-8](https://discourse.jupyter.org/t/jupyterhub-binder-workshop-in-oslo-sept-5-8) (17 posts)
* [Help us choose an updated &ldquo;Launch Binder&rdquo; badge!](https://discourse.jupyter.org/t/help-us-choose-an-updated-launch-binder-badge) (21 posts)
* [PyCon 2019 and mybinder.org](https://discourse.jupyter.org/t/pycon-2019-and-mybinder-org) (17 posts)
* [The Binder Federation](https://discourse.jupyter.org/t/the-binder-federation) (14 posts)
* [Microsoft Word Integration (Intern Project)](https://discourse.jupyter.org/t/microsoft-word-integration-intern-project) (17 posts)
* [Jupyter community workshop: proposal on diversity, inclusion and accessibility](https://discourse.jupyter.org/t/jupyter-community-workshop-proposal-on-diversity-inclusion-and-accessibility) (20 posts)
* [Sunset the GitHub repo jupyter/help?](https://discourse.jupyter.org/t/sunset-the-github-repo-jupyter-help) (17 posts)

## Active topics on the Community Forum

These are topics with lots of activity in recent history.

In [10]:
params = {"order": "posts", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most posts in recent history
posts = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(posts))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (103 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (59 posts)
* [Tip: embed custom github content in a Binder link with nbgitpuller](https://discourse.jupyter.org/t/tip-embed-custom-github-content-in-a-binder-link-with-nbgitpuller) (52 posts)
* [Would a &ldquo;The Littlest Binder&rdquo; be useful?](https://discourse.jupyter.org/t/would-a-the-littlest-binder-be-useful) (31 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (29 posts)
* [Plugins of considerations](https://discourse.jupyter.org/t/plugins-of-considerations) (28 posts)
* [A proposal for JupyterHub communications](https://discourse.jupyter.org/t/a-proposal-for-jupyterhub-communications) (26 posts)
* [Binder/BinderHub Workshops &ndash; Help Wanted!](https://discourse.jupyter.org/t/binder-binderhub-workshops-help-wanted) (25 posts)
* [Help us choose an updated &ldquo;Launch Binder&rdquo; badge!](https://discourse.jupyter.org/t/help-us-choose-an-updated-launch-binder-badge) (21 posts)
* [Creating a specification for reproducible repositories](https://discourse.jupyter.org/t/creating-a-specification-for-reproducible-repositories) (21 posts)

In [11]:
counts = counts_from_activity(posts)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:            category_id  bookmarked     parent_category  \
  11       accessibility           2      Special Topics   
  10      Admin requests           1       Site Feedback   
  0                 Meta           4                Meta   
  1        announcements           1                Meta   
  8           JupyterLab           1          JupyterLab   
  6              discuss           3          JupyterHub   
  2           JupyterHub           1          JupyterHub   
  9   Enterprise Gateway           2  Enterprise Gateway   
  3       Community Chat           3      Community Chat   
  7              discuss           6              Binder   
  4               Binder           4              Binder   
  5            BinderHub           2              Binder   
  
                            parent/category  
  11           Special Topics/accessibility  
  10           Site Feedback/Admin requests  
  0                               Meta/Meta  
  1                      Meta/announcements  
  8                   JupyterLab/JupyterLab  
  6                      JupyterHub/discuss  
  2                   JupyterHub/JupyterHub  
  9   Enterprise Gateway/Enterprise Gateway  
  3           Community Chat/Community Chat  
  7                          Binder/discuss  
  4                           Binder/Binder  
  5                        Binder/BinderHub  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['accessibility', 'Admin requests', 'Meta', 'announcements', 'JupyterLab', 'discuss', 'JupyterHub', 'Enterprise Gateway', 'Community Chat', 'discuss', 'Binder', 'BinderHub']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## Recently-created topics

These are topics that were recently created, sorted by the amount of activity
in each one.

In [12]:
params = {"order": "created", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Sort created by the most posted for recently-created posts
created = pd.DataFrame(resp.json()['topic_list']['topics'])
created = created.sort_values('posts_count', ascending=False)
Markdown(topics_to_markdown(created))

* [What&rsquo;s an alternative to the word &ldquo;Fellow&rdquo; or &ldquo;Fellowship&rdquo;?](https://discourse.jupyter.org/t/whats-an-alternative-to-the-word-fellow-or-fellowship) (12 posts)
* [Hiding code cell on launch](https://discourse.jupyter.org/t/hiding-code-cell-on-launch) (8 posts)
* [Binder workshop Sun. July 21, 9 am to 1 pm Swiss time: ~50 users](https://discourse.jupyter.org/t/binder-workshop-sun-july-21-9-am-to-1-pm-swiss-time-50-users) (5 posts)
* [Guidance for local Jupyter Days events](https://discourse.jupyter.org/t/guidance-for-local-jupyter-days-events) (5 posts)
* [Market Research: What is a JupyterHub?](https://discourse.jupyter.org/t/market-research-what-is-a-jupyterhub) (5 posts)
* [Is there a way to debug Jupyterlab .py code?](https://discourse.jupyter.org/t/is-there-a-way-to-debug-jupyterlab-py-code) (4 posts)
* [Is there a hook or signal when the tab closes or goes out of focus?](https://discourse.jupyter.org/t/is-there-a-hook-or-signal-when-the-tab-closes-or-goes-out-of-focus) (3 posts)
* [Webserver headers and ciphers](https://discourse.jupyter.org/t/webserver-headers-and-ciphers) (3 posts)
* [Best way to inject per user config for JuptyerHub](https://discourse.jupyter.org/t/best-way-to-inject-per-user-config-for-juptyerhub) (3 posts)
* [JupyterHUB on top of EMR and GPU nodes](https://discourse.jupyter.org/t/jupyterhub-on-top-of-emr-and-gpu-nodes) (3 posts)

In [13]:
counts = counts_from_activity(created)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:                         category_id  bookmarked parent_category  \
  8                               Q&A           2             Q&A   
  9                          Notebook           1        Notebook   
  1                              Meta           2            Meta   
  7                        JupyterLab           7      JupyterLab   
  0  Zero to JupyterHub on Kubernetes           0      JupyterHub   
  2                        JupyterHub           0      JupyterHub   
  5                           discuss           0      JupyterHub   
  3                    Community Chat           2  Community Chat   
  4                            Binder           1          Binder   
  6                           discuss           1          Binder   
  
                                 parent/category  
  8                                      Q&A/Q&A  
  9                            Notebook/Notebook  
  1                                    Meta/Meta  
  7                        JupyterLab/JupyterLab  
  0  JupyterHub/Zero to JupyterHub on Kubernetes  
  2                        JupyterHub/JupyterHub  
  5                           JupyterHub/discuss  
  3                Community Chat/Community Chat  
  4                                Binder/Binder  
  6                               Binder/discuss  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['Q&A', 'Notebook', 'Meta', 'JupyterLab', 'Zero to JupyterHub on Kubernetes', 'JupyterHub', 'discuss', 'Community Chat', 'Binder', 'discuss']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## User activity in the Community Forum

**Top posters**

These people have posted lots of comments, replies, answers, etc in the community forum.

In [14]:
def plot_user_data(users, column, sort=False):
    plt_data = users.sort_values(column, ascending=False).head(50)
    x = alt.X("username", sort=plt_data['username'].tolist()) if sort is True else 'username'
    ch = alt.Chart(data=plt_data).mark_bar().encode(
        x=x,
        y=column
    )
    return ch

In [15]:
url = "https://discourse.jupyter.org/directory_items.json"
params = {"period": "quarterly", "order": "post_count"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
users = pd.DataFrame(resp.json()['directory_items'])
users['username'] = users['user'].map(lambda a: a['username'])

In [16]:
plot_user_data(users.head(50), 'post_count')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             79    6          207             108         143        1422   
  1             92    3          207             127         137         968   
  2             67   24           16              54          74         670   
  3             91    8           12              65          57         807   
  4             60   12            9              43          46         801   
  5             80  116           27              58          45         634   
  6             20  220            1              29          37         305   
  7             68   73           16              22          27        1554   
  8             55   99           37              16          22         505   
  9             34  485           47              20          21         268   
  10            69   11           64              13          20         911   
  11            75   30           67              16          20         292   
  12            40  217           24               4          17         179   
  14            88   77            8              12          14         784   
  13            34  419            0               3          14          78   
  15            20  234            3              11          12          65   
  16            38  432           30               6          12         283   
  17            28   60            7               8          11         321   
  18            44  222            7              18          11         142   
  19            32  372            6              10          11         165   
  20             9  679            1              10           9          87   
  21            25   34            3              16           9         157   
  24            13  566            0               1           8          32   
  26             7  254            9               1           8          92   
  25            10  686            0               3           8          92   
  23             8  556            3               7           8          70   
  22            10  510            0               6           8          10   
  27            14  487            0               0           7          83   
  28            18  462            0               1           7          73   
  29            11  390            1               2           7          37   
  30            53  239            2               6           7         291   
  31            61  303            7               3           7         258   
  32            17   71            1               4           7         111   
  33            26  268            5              12           7         107   
  39            13  227            7               2           6         176   
  42            23    7           20               8           6         224   
  41            18    9            0               7           6          62   
  40            15  237            1              18           6          51   
  37            10  145            0               7           6          41   
  38            19  333            2               0           6          33   
  36             6  565            3               5           6          35   
  35             7  640            0               0           6          23   
  34            10  195            0               1           6          34   
  43             8  562            0               0           5          25   
  44            10  351            0               6           5          53   
  45            11  590            0               2           5          48   
  46            12  430            2               3           5          42   
  47            12  352            0               7           5          50   
  48            17  141           20               7           5  

**Forum users, sorted by likes given**

These are Community Forum members that "liked" other people's posts. We appreciate
anybody taking the time to tell someone else they like what they're shared!

In [17]:
plot_user_data(users.head(50), 'likes_given')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             79    6          207             108         143        1422   
  1             92    3          207             127         137         968   
  11            75   30           67              16          20         292   
  10            69   11           64              13          20         911   
  9             34  485           47              20          21         268   
  8             55   99           37              16          22         505   
  16            38  432           30               6          12         283   
  5             80  116           27              58          45         634   
  12            40  217           24               4          17         179   
  42            23    7           20               8           6         224   
  48            17  141           20               7           5         138   
  7             68   73           16              22          27        1554   
  2             67   24           16              54          74         670   
  3             91    8           12              65          57         807   
  4             60   12            9              43          46         801   
  26             7  254            9               1           8          92   
  14            88   77            8              12          14         784   
  31            61  303            7               3           7         258   
  39            13  227            7               2           6         176   
  18            44  222            7              18          11         142   
  17            28   60            7               8          11         321   
  19            32  372            6              10          11         165   
  33            26  268            5              12           7         107   
  21            25   34            3              16           9         157   
  23             8  556            3               7           8          70   
  36             6  565            3               5           6          35   
  15            20  234            3              11          12          65   
  38            19  333            2               0           6          33   
  46            12  430            2               3           5          42   
  30            53  239            2               6           7         291   
  32            17   71            1               4           7         111   
  6             20  220            1              29          37         305   
  29            11  390            1               2           7          37   
  40            15  237            1              18           6          51   
  20             9  679            1              10           9          87   
  43             8  562            0               0           5          25   
  44            10  351            0               6           5          53   
  45            11  590            0               2           5          48   
  41            18    9            0               7           6          62   
  47            12  352            0               7           5          50   
  25            10  686            0               3           8          92   
  37            10  145            0               7           6          41   
  35             7  640            0               0           6          23   
  34            10  195            0               1           6          34   
  28            18  462            0               1           7          73   
  27            14  487            0               0           7          83   
  24            13  566            0               1           8          32   
  22            10  510            0               6           8          10   
  13            34  419            0               3          14  

**Forum users, sorted by likes received**

These are folks that posted things other people in the Community Forum liked.

In [18]:
plot_user_data(users.head(50), 'likes_received')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  1             92    3          207             127         137         968   
  0             79    6          207             108         143        1422   
  3             91    8           12              65          57         807   
  5             80  116           27              58          45         634   
  2             67   24           16              54          74         670   
  4             60   12            9              43          46         801   
  6             20  220            1              29          37         305   
  7             68   73           16              22          27        1554   
  9             34  485           47              20          21         268   
  18            44  222            7              18          11         142   
  40            15  237            1              18           6          51   
  8             55   99           37              16          22         505   
  21            25   34            3              16           9         157   
  11            75   30           67              16          20         292   
  10            69   11           64              13          20         911   
  14            88   77            8              12          14         784   
  33            26  268            5              12           7         107   
  15            20  234            3              11          12          65   
  19            32  372            6              10          11         165   
  20             9  679            1              10           9          87   
  42            23    7           20               8           6         224   
  17            28   60            7               8          11         321   
  23             8  556            3               7           8          70   
  47            12  352            0               7           5          50   
  41            18    9            0               7           6          62   
  48            17  141           20               7           5         138   
  37            10  145            0               7           6          41   
  22            10  510            0               6           8          10   
  44            10  351            0               6           5          53   
  16            38  432           30               6          12         283   
  30            53  239            2               6           7         291   
  36             6  565            3               5           6          35   
  32            17   71            1               4           7         111   
  12            40  217           24               4          17         179   
  46            12  430            2               3           5          42   
  25            10  686            0               3           8          92   
  31            61  303            7               3           7         258   
  13            34  419            0               3          14          78   
  39            13  227            7               2           6         176   
  29            11  390            1               2           7          37   
  45            11  590            0               2           5          48   
  34            10  195            0               1           6          34   
  28            18  462            0               1           7          73   
  26             7  254            9               1           8          92   
  24            13  566            0               1           8          32   
  49             6  699            0               1           5          39   
  35             7  640            0               0           6          23   
  38            19  333            2               0           6          33   
  43             8  562            0               0           5  

In [19]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>